In [1]:
%matplotlib inline
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#### Motivating Support Vector Machines


In [ ]:
from sklearn.datasets import make_blobs
X,y=make_blobs(n_samples=50, centers=2,
              random_state=0, cluster_std=0.60)
plt.scatter(X[:,0],X[:,1],c=y,s=50,cmap='viridis');

In [ ]:
Xfit=np.linspace(-1,3.5)
plt.scatter(X[:,0],X[:,1],c=y,s=50,cmap='viridis')
plt.plot([0.6],[2.1],'x',color='red',markeredgewidth=2,markersize=10)

for m,b in [(1,0.65),(0.5,1.6),(-0.2,2.9)]:
    plt.plot(Xfit,m*Xfit+b,'-k')

plt.xlim(-1,3.5);

### Support Vector Machines: Maximizing the Margin

In [ ]:
Xfit=np.linspace(-1,3.5)
plt.scatter(X[:,0],X[:,1],c=y, s=50, cmap='viridis')

for m,b,d in[(1,0.65,0.33),(0.5,1.6,0.55),(-0.2,2.9,0.2)]:
    yfit=m*Xfit+b
    plt.plot(Xfit,yfit,'-k')
    plt.fill_between(Xfit,yfit-d,yfit+d, edgecolor='none',
                    color='#AAAAAA', alpha=0.4)
plt.xlim(-1,3.5);

#### Fitting a support vector machine

In [ ]:
from sklearn.svm import SVC#"support vector classifier"
model=SVC(kernel='linear',C=1E10)
model.fit(X,y)

In [ ]:
#create a function that will plot the SVM decision boundaries 
def plot_svc_function(model,ax=None, plot_support=True):
        """Plot the decision funtion for a two-dimensional SVC"""
        
        if ax is None:
            ax=plt.gca()
        xlim=ax.get_xlim()
        ylim=ax.get_ylim()
        
        #create grid to evealuate model
        x=np.linspace(xlim[0],xlim[1],30)
        y=np.linspace(ylim[0],ylim[1],30)
        Y,X=np.meshgrid(y,x)
        xy=np.vstack([X.ravel(),Y.ravel()]).T
        p=model.decision_function(xy).reshape(X.shape)
        
        #plot decision boundaries and margin
        ax.contour(X,Y,p,colors='k',
                  levels=[-1,0,1], alpha=0.5,
                  linestyles=['--','-','--'])
        
        #plot support vectors
        if plot_support:
            ax.scatter(model.support_vectors_[:,0],
                      model.support_vectors_[:,1],
                      s=300, linewidth=1, facecolors='none');
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)

In [ ]:
plt.scatter(X[:,0],X[:,1], c=y,s=50, cmap='viridis')
plot_svc_function(model);

In [ ]:
model.support_vectors_


In [ ]:
#a model that learned from the first 60 points and first 120 points of this dataset
def plot_svm(N=10,ax=None):
    X,y=make_blobs(n_samples=200,n_features=2,centers=2,
                  random_state=0,cluster_std=0.60)
    X=X[:N]
    y=y[:N]
    model=SVC(kernel='linear',C=1E10)
    model.fit(X,y)
    
    ax=ax or plt.gca()
    ax.scatter(X[:,0],X[:,1],c=y,s=50, cmap='viridis')
    ax.set_xlim(-1,4)
    ax.set_ylim(-1,6)
    plot_svc_function(model,ax)
    
fig, ax=plt.subplots(1,2, figsize=(16,6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)
for axi,N in zip(ax,[60,120]):
    plot_svm(N,axi)
    axi.set_title('N={0}'.format(N))

In [ ]:
from ipywidgets import interact,fixed
interact(plot_svm,N=[10,200],ax=fixed(None));

##### Beyond linear boundaries:Kernel SVM

In [ ]:
from sklearn.datasets import make_circles
X,y=make_circles(100,factor=.1,noise=.1)

clf=SVC(kernel='linear').fit(X,y)

plt.scatter(X[:,0],X[:,1],c=y,s=50, cmap='viridis')
plot_svc_function(clf,plot_support=False);

In [ ]:
r=np.exp(-(X**2).sum(1))

In [ ]:
from mpl_toolkits import mplot3d

def plot_3D(elev=30,azim=30,X=X, y=y):
    ax=plt.subplot(projection='3d')
    ax.scatter3D(X[:,0],X[:,1],r,c=y,s=50,cmap='viridis')
    ax.view_init(elev=elev,azim=azim)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('r')

interact(plot_3D,X=fixed(X),y=fixed(y));

In [ ]:
clf=SVC(kernel='rbf',C=1E6)
clf.fit(X,y)

In [ ]:
plt.scatter(X[:,0],X[:,1],c=y,s=50,cmap='viridis')
plot_svc_function(clf)
plt.title('Kernel SVM fit')
plt.scatter(clf.support_vectors_[:,0],clf.support_vectors_[:,1],
           s=300,lw=1,facecolors='none');

#### Tuning the SVM: Softening margins

In [ ]:
X,y=make_blobs(n_samples=100,centers=2,
              random_state=0, cluster_std=1.2)
plt.scatter(X[:,0],X[:,1],c=y,s=50,cmap='viridis');

In [ ]:
X,y=make_blobs(n_samples=100,centers=2,
              random_state=0, cluster_std=0.8)

fig,ax=plt.subplots(1,2, figsize=(16,9))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for axi,C in zip(ax,[10.0,0.1]):
    model=SVC(kernel='linear',C=C).fit(X,y)
    axi.scatter(X[:,0],X[:,1],c=y,s=50,cmap='viridis')
    plot_svc_function(model,axi)
    axi.scatter(model.support_vectors_[:,0],
               model.support_vectors_[:,1],
               s=300, lw=1, facecolors='none');
    
    axi.set_title('C={0:.1f}'.format(C),size=14)

### Example: Face Recognition

In [ ]:
from sklearn.datasets import fetch_lfw_people
faces=fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)

In [ ]:
fig,ax=plt.subplots(3,5)
for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i],cap='bone')
    axi.set(xticks=[],yticks=[],
           xlabel=faces.target_names[faces.target[i]])

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline

pca=RandomizedPCA(n_components=150,whiten=True, random_state=42)
svc=SVC(kernel='rbf',class_weight='balanced')
model=make_pipeline(pca,svc)

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest=train_test_split(faces.data,faces.target,
                                          random_state=42)

In [ ]:
from sklearn.model_selection  import GridSearchCV
param_grid={'svc__c':[1,5,10,50],
          'svc__gamma':[0.0001,0.0005,0.001,0.005]}
grid=GridSearchCV(model,param_grid)
%time grid.fit(Xtrain,ytrain)
print(grid.best_params_)

In [ ]:
model=grid.best_estimator_
yfit=model.predict(Xtest)

In [ ]:
fig,ax=plt.subplots(4,6)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(62,47),cmap='bone')
    axi.set(xticks=[],yticks=[])
    axi.set_ylabel(faces.target_names[yfit[i]].split()[-1],
                  color='black' if yfit[i]==ytest[i] else 'red')
    fig.suptitle('Predicted Names: Incorrect labels in red', size=14);

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest,yfit,
                           target_names=faces.target_names))

In [ ]:
from sklearn.metrics import confusion_matrix
mat=confusion_matrix(ytest,yfit)
sns.heatmap(mat.T,square=True, annot=True, fmt='d', cbar=False,
           xticklabels=faces.target_names,
           yticklabels=faces.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label');